In [ ]:
from data.procesing import ParseTaggedDataset
directory = "256-chunk-length-dataset"
ParseTaggedDataset(directory)

In [1]:
directory = "256-chunk-length-dataset"

In [2]:
from data.procesing import StreamingSongDataset, ParseTaggedDataset
directory = "256-chunk-length-dataset"
dataset = StreamingSongDataset(f"E:\\SongsDataset\\{directory}\\data", f"E:\\SongsDataset\\{directory}\\labels")

In [3]:
from torch.utils.data import random_split, DataLoader
import torch

generator = torch.Generator().manual_seed(42)

train_len = int(len(dataset) * 0.9)
train_set, test_set = random_split(dataset, [train_len, len(dataset) - train_len], generator=generator)

train_dataloader = DataLoader(
    train_set,
    batch_size=32,
    shuffle=True,  # Already shuffled
    num_workers=4,
    prefetch_factor=10,
    pin_memory=True,
)

test_dataloader = DataLoader(
test_set,
batch_size=32,
shuffle=True,  # Already shuffled
num_workers=4,
prefetch_factor=10,
pin_memory=True,
)

In [4]:
class Config:
    # === General ===
    model_name = "Transformer-Classifier"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.float32
    save_path = f"E:\\Coding\\SongAnalyzer\\Analyzer\\src\\trained_models\\{model_name}\\"
    seed = 42

    # === Training ===
    num_epochs = 30
    batch_size = 24
    learning_rate = 5e-5
    weight_decay = 1e-5
    warmup_percent = 0.15
    max_grad_norm = 1.0
    log_every = 10  # steps between logs (optional)
    save_checkpoints = True

    # === Dataset ===
    use_masks = True
    num_workers = 4
    val_split = 0.2
    shuffle = True

    # === Model Behavior ===
    variational = False
    autoregressive = False
    #class_ratio = torch.tensor(class_ratio).to(device="cuda")

    # === Loss Coefficients ===
    beta_schedule = "log"   # e.g., "log", "linear", etc. (for getBetaLog)
    beta_max = 1.0
    cycle_length = 2
    contrastive_coeff = 0.1  # if using contrastive loss
    margin = 0.1

In [5]:
import torch.optim as optim
from datasets import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

def train(model, test_dataloader, train_dataloader, config):
    # Training setup
    criterion = nn.BCEWithLogitsLoss()#pos_weight=config.class_ratio)
    optimizer = optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-5)  # L2 regularization
    model.to("cuda")

    # Training loop
    for epoch in range(40):
        train_loss_total = 0
        for batch in tqdm(train_dataloader):
            inputs, labels = batch

            inputs = inputs.to("cuda")

            outputs = model(inputs)
            loss = criterion(outputs, genre_labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_total += loss.item()

        test_loss_average, correct, total, all_preds, all_labels = eval(model, test_dataloader, criterion)

        precision_macro = precision_score(all_labels, all_preds, average='macro')
        recall_macro = recall_score(all_labels, all_preds, average='macro')
        f1_macro = f1_score(all_labels, all_preds, average='macro')

        train_loss_average = train_loss_total / len(train_dataloader)

        print(f"Epoch {epoch + 1}, Train Loss: {train_loss_average:.4f}")
        print(f"Epoch {epoch + 1}, Test Loss: {test_loss_average:.4f}")
        print(f"Test Accuracy: {correct / total:.2%}")
        print(f"Precision: {precision_macro:.4f}\t Recall: {recall_macro:.4f}\t F1: {f1_macro:.4f}")

        torch.save(model, f".\\{config.path}\\Classifier-Epoch-{epoch + 1}.pt")


def eval(model, dataloader, criterion):
    test_loss_total = 0
    correct = 0
    total = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader):
            inputs, labels = batch

            inputs = inputs.to("cuda")

            outputs = model(inputs)
            loss = criterion(outputs, genre_labels)

            test_loss_total += loss.item()

            preds = (outputs.sigmoid() > 0.2).int()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return test_loss_total / len(dataloader), correct, total, all_preds, all_labels

In [6]:
import numpy as np

def model_size(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    return sum([np.prod(p.size()) for p in model_parameters])

In [7]:
import torch.nn as nn
from torchvision.models import resnet50

# Resnet-50
# Define the model
model = resnet50(pretrained=True)

# Replace the last layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 249)
model_size(model)

E:\Coding\SongAnalyzer\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
E:\Coding\SongAnalyzer\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


24018233

In [8]:
train(model, train_dataloader, train_dataloader, Config)

  0%|          | 0/5 [00:01<?, ?it/s]

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "E:\Coding\SongAnalyzer\.venv\lib\site-packages\torch\utils\data\_utils\worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "E:\Coding\SongAnalyzer\.venv\lib\site-packages\torch\utils\data\_utils\fetch.py", line 50, in fetch
    data = self.dataset.__getitems__(possibly_batched_index)
  File "E:\Coding\SongAnalyzer\.venv\lib\site-packages\torch\utils\data\dataset.py", line 420, in __getitems__
    return [self.dataset[self.indices[idx]] for idx in indices]
  File "E:\Coding\SongAnalyzer\.venv\lib\site-packages\torch\utils\data\dataset.py", line 420, in <listcomp>
    return [self.dataset[self.indices[idx]] for idx in indices]
  File "E:\Coding\SongAnalyzer\Analyzer\src\data\procesing.py", line 38, in __getitem__
    song = torch.load(song_path, map_location='cpu')  # memory-mapped access
  File "E:\Coding\SongA